# Elmo + Embeddings

Probemos si usando también los embeddings de fastText obtenemos algo razonable...



In [1]:
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
import random
import torch

torch.manual_seed(2019)
np.random.seed(2019)
tf.random.set_random_seed(2019)
random.seed(2019)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_test = pd.read_table("../../../data/es/reference_es.tsv", header=None, 
                        names=["text", "HS", "TR", "AG"], quoting=csv.QUOTE_NONE)


text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]
text_test, y_test = df_test["text"], df_test["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))
print("Instancias de test: {}".format(len(df_test)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500
Instancias de test: 1600


In [2]:

df_eyeball = df_dev.sample(n=300, random_state=20190911)
text_eyeball, y_eyeball = df_eyeball["text"], df_eyeball["HS"]


In [3]:
import fastText
import os


model = fastText.load_model(os.path.expanduser("../../../WordVectors/UBA_w3_300.bin"))

Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [4]:
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.sequence import pad_sequences


max_length = 40

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

def preprocess_tweet(tweet):
    tokens = tokenizer.tokenize(tweet)
    
    if len(tokens) >= max_length:
        tokens = tokens[:max_length]
    else:
        tokens = tokens + [''] * (max_length - len(tokens))
    return tokens


tokens_train = [preprocess_tweet(tweet) for tweet in df_train["text"].values]
tokens_dev = [preprocess_tweet(tweet) for tweet in df_dev["text"].values]
tokens_test = [preprocess_tweet(tweet) for tweet in df_test["text"].values]

Using TensorFlow backend.


In [5]:
%%capture
from elmoformanylangs import Embedder

e = Embedder("../../../models/elmo/es/")

Carguemos embeddings

In [6]:
import numpy as np


X_train = np.array(e.sents2elmo(tokens_train))
X_dev = np.array(e.sents2elmo(tokens_dev))
X_test = np.array(e.sents2elmo(tokens_test))

In [7]:
def get_embeddings(toks):
    ret = []
    
    for tok in toks:
        vec = model.get_word_vector(tok)
        ret.append(vec)
    return ret

X_emb_train = np.array([get_embeddings(toks) for toks in tokens_train])
X_emb_dev = np.array([get_embeddings(toks) for toks in tokens_dev])
X_emb_test = np.array([get_embeddings(toks) for toks in tokens_test])

X_emb_train.shape, X_emb_dev.shape, X_emb_test.shape

((4500, 40, 300), (500, 40, 300), (1600, 40, 300))

In [8]:
from keras.models import Model
from keras.layers import Dense, Embedding, Dropout, CuDNNLSTM, CuDNNGRU, Input, Concatenate, Bidirectional, GlobalMaxPool1D
from keras.optimizers import Adam
from keras.preprocessing import sequence

embedding_dim = 1024

optimizer_args = {
    "lr": 0.00075,
    "decay": 0.01,
}

elmo_input = Input(shape=X_train[0].shape)
emb_input = Input(shape=X_emb_train[0].shape)

x = Concatenate()([elmo_input, emb_input])
x = Bidirectional(CuDNNLSTM(256))(x)
x = Dropout(0.65)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[elmo_input, emb_input], outputs=[output])

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
print(model.summary())



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 1024)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 40, 300)      0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 40, 1324)     0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 512)          3239936     concatenate_1[0][0]              
__________

In [9]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint('/tmp/lstm_model.h5', save_best_only=True, monitor='val_acc', verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', patience=30)


model.fit([X_train, X_emb_train], y_train, 
          callbacks=[checkpointer, early_stopper],
          validation_data=([X_dev, X_emb_dev], y_dev), epochs=100, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - 6s 1ms/step - loss: 0.6433 - acc: 0.6456 - val_loss: 0.6010 - val_acc: 0.6580

Epoch 00001: val_acc improved from -inf to 0.65800, saving model to /tmp/lstm_model.h5
Epoch 2/100
4500/4500 [==============================] - 3s 557us/step - loss: 0.5106 - acc: 0.7502 - val_loss: 0.4981 - val_acc: 0.7840

Epoch 00002: val_acc improved from 0.65800 to 0.78400, saving model to /tmp/lstm_model.h5
Epoch 3/100
4500/4500 [==============================] - 2s 540us/step - loss: 0.4216 - acc: 0.8036 - val_loss: 0.4436 - val_acc: 0.8000

Epoch 00003: val_acc improved from 0.78400 to 0.80000, saving model to /tmp/lstm_model.h5
Epoch 4/100
4500/4500 [==============================] - 2s 551us/step - loss: 0.3687 - acc: 0.8382 - val_loss: 0.4183 - val_acc: 0.8260

Epoch 00004: val_acc improved from 0.80000 to 0.82600, saving model to /tmp/lstm_model.h5
Epoch 5/100
4500/4500 [=======================

In [10]:
from hate.utils import print_evaluation
print("biLSTM - Elmo+Embeddings -- \n\n")
print("Evaluación sobre dev")

model.load_weights(checkpointer.filepath)

print_evaluation(model, [X_dev, X_emb_dev], y_dev)
print("\nEvaluación sobre test")

print_evaluation(model, [X_test, X_emb_test], y_test)

biLSTM - Elmo+Embeddings -- 


Evaluación sobre dev
500/500 [==============================] - 0s 262us/step
Loss           : 0.4361
Accuracy       : 0.8380
Precision(1)   : 0.8249
Precision(1)   : 0.8481
Precision(avg) : 0.8365

Recall(1)      : 0.8063
Recall(0)      : 0.8633
Recall(avg)    : 0.8348

F1(1)          : 0.8155
F1(0)          : 0.8556
F1(avg)        : 0.8356

Evaluación sobre test
1600/1600 [==============================] - 0s 261us/step
Loss           : 0.6437
Accuracy       : 0.7312
Precision(1)   : 0.6554
Precision(1)   : 0.7965
Precision(avg) : 0.7260

Recall(1)      : 0.7348
Recall(0)      : 0.7287
Recall(avg)    : 0.7318

F1(1)          : 0.6929
F1(0)          : 0.7611
F1(avg)        : 0.7270


## GRU

In [11]:
embedding_dim = 1024

optimizer_args = {
    "lr": 0.0075,
    "decay": 0.01,
}

elmo_input = Input(shape=X_train[0].shape)
emb_input = Input(shape=X_emb_train[0].shape)

x = Concatenate()([elmo_input, emb_input])
x = Bidirectional(CuDNNGRU(256))(x)
x = Dropout(0.65)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[elmo_input, emb_input], outputs=[output])

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
print(model.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 40, 1024)     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 40, 300)      0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 40, 1324)     0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 512)          2429952     concatenate_2[0][0]              
__________

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint('/tmp/lstm_model.h5', save_best_only=True, monitor='val_acc', verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', patience=30)


model.fit([X_train, X_emb_train], y_train, 
          callbacks=[checkpointer, early_stopper],
          validation_data=([X_dev, X_emb_dev], y_dev), epochs=100, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - 3s 755us/step - loss: 0.8181 - acc: 0.5727 - val_loss: 0.6239 - val_acc: 0.6380

Epoch 00001: val_acc improved from -inf to 0.63800, saving model to /tmp/lstm_model.h5
Epoch 2/100
4500/4500 [==============================] - 2s 504us/step - loss: 0.6306 - acc: 0.6631 - val_loss: 0.5369 - val_acc: 0.7440

Epoch 00002: val_acc improved from 0.63800 to 0.74400, saving model to /tmp/lstm_model.h5
Epoch 3/100
4500/4500 [==============================] - 2s 527us/step - loss: 0.5390 - acc: 0.7369 - val_loss: 0.4965 - val_acc: 0.7660

Epoch 00003: val_acc improved from 0.74400 to 0.76600, saving model to /tmp/lstm_model.h5
Epoch 4/100
4500/4500 [==============================] - 2s 511us/step - loss: 0.5008 - acc: 0.7578 - val_loss: 0.4906 - val_acc: 0.7520

Epoch 00004: val_acc did not improve from 0.76600
Epoch 5/100
4500/4500 [==============================] - 2s 524us/step - loss: 0.477

In [13]:
from hate.utils import print_evaluation
print("biGRU - Elmo+Embeddings -- \n\n")
print("Evaluación sobre dev")

model.load_weights(checkpointer.filepath)

print_evaluation(model, [X_dev, X_emb_dev], y_dev)
print("\nEvaluación sobre test")

print_evaluation(model, [X_test, X_emb_test], y_test)


biGRU - Elmo+Embeddings -- 


Evaluación sobre dev
500/500 [==============================] - 0s 265us/step
Loss           : 0.4295
Accuracy       : 0.8140
Precision(1)   : 0.7919
Precision(1)   : 0.8315
Precision(avg) : 0.8117

Recall(1)      : 0.7883
Recall(0)      : 0.8345
Recall(avg)    : 0.8114

F1(1)          : 0.7901
F1(0)          : 0.8330
F1(avg)        : 0.8116

Evaluación sobre test
1600/1600 [==============================] - 0s 269us/step
Loss           : 0.5484
Accuracy       : 0.7444
Precision(1)   : 0.6816
Precision(1)   : 0.7921
Precision(avg) : 0.7369

Recall(1)      : 0.7136
Recall(0)      : 0.7660
Recall(avg)    : 0.7398

F1(1)          : 0.6973
F1(0)          : 0.7788
F1(avg)        : 0.7380


## GRU + Global Max Pooling

In [14]:
from keras.models import Model
from keras.layers import Dense, Embedding, Dropout, CuDNNLSTM, CuDNNGRU, Input, Concatenate, Bidirectional, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.preprocessing import sequence

embedding_dim = 1024

optimizer_args = {
    "lr": 0.00075,
    "decay": 0.01,
}

elmo_input = Input(shape=X_train[0].shape)
emb_input = Input(shape=X_emb_train[0].shape)

x = Concatenate()([elmo_input, emb_input])
x = Bidirectional(CuDNNGRU(256, return_sequences=True))(x)
x = Dropout(0.60)(x)
x = GlobalMaxPooling1D()(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[elmo_input, emb_input], outputs=[output])

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
print(model.summary())



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 40, 1024)     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 40, 300)      0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 40, 1324)     0           input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 40, 512)      2429952     concatenate_3[0][0]              
__________

In [15]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint('/tmp/lstm_model.h5', save_best_only=True, monitor='val_acc', verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', patience=30)


model.fit([X_train, X_emb_train], y_train, 
          callbacks=[checkpointer, early_stopper],
          validation_data=([X_dev, X_emb_dev], y_dev), epochs=100, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - 4s 782us/step - loss: 0.5980 - acc: 0.7142 - val_loss: 0.5232 - val_acc: 0.7580

Epoch 00001: val_acc improved from -inf to 0.75800, saving model to /tmp/lstm_model.h5
Epoch 2/100
4500/4500 [==============================] - 2s 528us/step - loss: 0.3903 - acc: 0.8253 - val_loss: 0.4902 - val_acc: 0.7860

Epoch 00002: val_acc improved from 0.75800 to 0.78600, saving model to /tmp/lstm_model.h5
Epoch 3/100
4500/4500 [==============================] - 2s 524us/step - loss: 0.3221 - acc: 0.8667 - val_loss: 0.4804 - val_acc: 0.7900

Epoch 00003: val_acc improved from 0.78600 to 0.79000, saving model to /tmp/lstm_model.h5
Epoch 4/100
4500/4500 [==============================] - 2s 521us/step - loss: 0.2839 - acc: 0.8924 - val_loss: 0.4713 - val_acc: 0.8020

Epoch 00004: val_acc improved from 0.79000 to 0.80200, saving model to /tmp/lstm_model.h5
Epoch 5/100
4500/4500 [=====================

In [16]:
from hate.utils import print_evaluation
print("biGRU + MaxPool1D - Elmo+Embeddings -- \n\n")
print("Evaluación sobre dev")

model.load_weights(checkpointer.filepath)

print_evaluation(model, [X_dev, X_emb_dev], y_dev)
print("\nEvaluación sobre test")

print_evaluation(model, [X_test, X_emb_test], y_test)

biGRU + MaxPool1D - Elmo+Embeddings -- 


Evaluación sobre dev
500/500 [==============================] - 0s 271us/step
Loss           : 0.4126
Accuracy       : 0.8280
Precision(1)   : 0.8238
Precision(1)   : 0.8310
Precision(avg) : 0.8274

Recall(1)      : 0.7793
Recall(0)      : 0.8669
Recall(avg)    : 0.8231

F1(1)          : 0.8009
F1(0)          : 0.8486
F1(avg)        : 0.8248

Evaluación sobre test
1600/1600 [==============================] - 0s 289us/step
Loss           : 0.5544
Accuracy       : 0.7388
Precision(1)   : 0.6609
Precision(1)   : 0.8078
Precision(avg) : 0.7343

Recall(1)      : 0.7530
Recall(0)      : 0.7287
Recall(avg)    : 0.7409

F1(1)          : 0.7040
F1(0)          : 0.7662
F1(avg)        : 0.7351


## Error Analysis

Vamos a ver los tweets con mayores errores

In [17]:
df_dev["proba"] = model.predict([X_dev, X_emb_dev])


true_positives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] >= 0.5)].copy()
true_negatives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] < 0.5)].copy()

false_positives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] > 0.5)].copy()
false_positives.sort_values("proba", ascending=False, inplace=True)


false_negatives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] < 0.5)].copy()
false_negatives.sort_values("proba", ascending=True, inplace=True)

conf_matrix = pd.DataFrame([
    {"real":"hs=1", "pred_true": len(true_positives), "pred_false": len(false_negatives)},
    {"real":"hs=0", "pred_true": len(false_positives), "pred_false": len(true_negatives)}
])



conf_matrix.set_index("real", inplace=True)

print("Falsos negativos: {}".format(len(false_negatives)))
print("Falsos positivos: {}".format(len(false_positives)))

conf_matrix[["pred_true", "pred_false"]]

Falsos negativos: 49
Falsos positivos: 37


,pred_true,pred_false
real,,
hs=1,173,49
hs=0,37,241


## Proporción de Agresivos

In [18]:
print("Proporción de agresivos :", sum(df_dev["AG"] == 0) / len(df_dev))

hs = df_dev[df_dev["HS"] == 1]

print("Correlación AG - TR:", hs["AG"].corr(hs["TR"]))

Proporción de agresivos : 0.648
Correlación AG - TR: 0.42040512740887925


## Falsos negativos

In [19]:
pd.set_option('max_colwidth', 300)



print("Totales = ", len(false_negatives))
print("No AG ({}) AG ({})".format(sum(false_negatives["AG"] == 0), sum(false_negatives["AG"] == 1)))
false_negatives[["text", "proba", "HS", "AG"]]

Totales =  49
No AG (16) AG (33)


,text,proba,HS,AG
id,,,,
24126,LA SOLUCIÓN ES SOLDARLE EL TIMÓN DE LA PATERA RUMBO DE DONDE SALIERON CON AGUA Y COMIDA. https://t.co/59pInBdgnR,0.022556,1,0
22065,| lo único que dominas es mi polla en tu cara. https://t.co/qlYOd4zPwW,0.023897,1,1
24590,"SOMOS MUCHO MÁS DE LO QUE SUEÑAS SER PINCHÉ SUDACA CAGADO, y si no te gusta dime donde nos topamos pa reventarnos el hocico 1:1 @amarchesin1 https://t.co/vHHPBKS1AK",0.028093,1,1
24925,"La verdad que si, no veia la hora de llegar a los 5 mil amigos😂 ahora de a poco voy borrando esos indios indocumentados😂 https://t.co/Xw8q4GLM8v",0.048991,1,0
24864,rosalia me podría decir sudaca i wouldn't give a fuck,0.050993,1,1
20307,tu eres lesbi Ana xk no as probaddo mi polla,0.055418,1,1
23688,"Ninguna mujer es tan santa como para no aprender a coger bien, ni tan puta como para no aprender a amar.",0.082030,1,1
22560,@CarlaEUrena @FelipeCalderon Callateeeee !! Puta gallina cacarea !! Ni idea tienes de nada .,0.087284,1,1
22008,"@Namurii @medicaster_xz @Yorha2d @KeloKlau_ Se merecía de puta y no quiere que le metan la verga, jajaja no mamen",0.088236,1,0


## Falsos Positivos

In [20]:
pd.set_option('max_colwidth', 300)



print("Totales = ", len(false_positives))
print("No AG ({}) AG ({})".format(sum(false_positives["AG"] == 0), sum(false_positives["AG"] == 1)))
false_positives[["text", "proba", "HS", "AG"]]

Totales =  37
No AG (37) AG (0)


,text,proba,HS,AG
id,,,,
22457,"De Colombia tocará emigrar en pateras, como hacen los africanos subsaharianos.",0.984295,0,0
20376,"Zorra tu abuela, imbécil.",0.953888,0,0
21892,@saancisco Callate puta tu eres el traidor.,0.953812,0,0
21497,Talvez quede como zorra pero te lo mereces Socia😎,0.944706,0,0
20148,Racismo es que hagas enfrijoladas con frijoles güeros y no negros.,0.942656,0,0
24402,zorra eres tu que no te sientes hombre y no entiendes que toda mujer es bella,0.934405,0,0
22014,Ni tú eres una zorra hambrienta ni yo un incorregible depredador.,0.926217,0,0
22416,@Pepe_ElEzequiel Puta!? Puta con las que vos te metes Esa mujer es un ángel enviado por Jesucristo nuestro señor para darnos esa paz que buscamos sólo con verla,0.896160,0,0
22115,"Una chica no es una guarra, una zorra, una puta, una buscan por su manera de vestir.",0.886693,0,0
